In [26]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import datetime

In [27]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [28]:
rm -rf ./logs/

In [29]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [30]:
train_path = '/Users/amitaflalo/Desktop/deepnav/data/train/'
test_path = '/Users/amitaflalo/Desktop/deepnav/data/test/'
interval = 50

In [31]:
def file_select_train(name):  
    global accelometer
    global gyroscope
    global speed
    global df

    df = pd.read_csv(train_path + name , header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    speed = df.iloc[:, 9:10]

In [32]:
def file_select_test(name):  
    global accelometer
    global gyroscope
    global speed
    global df

    df = pd.read_csv(test_path + name , header= None)
    accelometer = df.iloc[:, 2:5] 
    gyroscope = df.iloc[:, 5:8] 
    speed = df.iloc[:, 9:10]

In [33]:
def get_batch(batch_size, index):

    global accelometer
    global gyroscope
    global speed
    global df
    
    idx = index

    #input to Network[accelometer, gyroscope  data]
    ##########################################################################################

    batch_accelometer = [accelometer.iloc[i * interval : i * interval + interval,:] for i in range(idx,idx + batch_size)]
    batch_accelometer = np.array(batch_accelometer, dtype='float64')

    batch_gyroscope = [gyroscope.iloc[i * interval : i * interval + interval,:] for i in range(idx,idx + batch_size)]
    batch_gyroscope = np.array(batch_gyroscope, dtype='float64')

    batch_sensors = np.concatenate((batch_accelometer,batch_gyroscope), axis = 2)

    #ground truth[acceleraion(m/s)]
    ########################################################################################## 

    batch_acceleraion = [speed.iloc[i * interval,:] for i in range(idx + 1,idx + batch_size + 1)]

    batch_acceleraion = np.array(batch_acceleraion,  dtype='float64')




    return batch_sensors, batch_acceleraion

In [34]:
file_select_train('locationData.csv')

In [35]:
x,y = get_batch(50,0)

In [36]:
x.shape

(50, 50, 6)

In [37]:
x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
# x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
# x = tf.keras.layers.MaxPool1D(3)(x)
# x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
# x = tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, activation='relu')(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)


In [38]:
x.shape

TensorShape([50, 48, 128])

In [73]:
def build_model():
  model = tf.keras.Sequential([
    tf.keras.layers.LayerNormalization(axis=[1, 2], input_shape = (50,6)),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 6, activation='relu', input_shape = (50,6)),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 6, activation='relu'),
    tf.keras.layers.MaxPool1D(3),

    tf.keras.layers.Conv1D(filters = 64, kernel_size = 6, activation='relu'),
    tf.keras.layers.Conv1D(filters = 64, kernel_size = 6, activation='relu'),


    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1)
  ])

  return model
model = build_model()
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_normalization_2 (Layer (None, 50, 6)             600       
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 45, 64)            2368      
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 40, 64)            24640     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 8, 64)             24640     
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 3, 64)             24640     
_________________________________________________________________
flatten_6 (Flatten)          (None, 192)              

In [74]:
def compute_loss(labels, logits):
  mse = tf.keras.losses.MeanSquaredError()
  loss = mse(labels, logits)
  return loss

def precent_loss(labels, logits):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    loss = mae(labels, logits)
    return loss

In [75]:
# Define our metrics for tensorboard
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

In [76]:
### Hyperparameter setting and optimization ###

# Optimization parameters:
EPOCHS = 200  # Increase this to train longer
batch_size = 10  # Experiment between 1 and 64
learning_rate = 1e-4  # Experiment between 1e-5 and 1e-1

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

In [77]:
optimizer = tf.keras.optimizers.Adam(learning_rate)

@tf.function
def train_step(x, y): 
  global loss_real
  # Use tf.GradientTape()
  with tf.GradientTape() as tape:
      y_hat = model(x) 
      loss = compute_loss(y, y_hat)

  # Now, compute the gradients 
  grads = tape.gradient(loss, model.trainable_variables) 
  # Apply the gradients to the optimizer so it can update the model accordingly
  optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
  #metrics
  train_loss(loss)

@tf.function
def test_step(x, y): 
    y_hat = model(x) 
    loss = precent_loss(y, y_hat)
    test_loss(loss)

In [78]:
##################
# Begin training!#
##################

for epoch in range(EPOCHS):

  # Grab a batch and propagate it through the network
  for filename in os.listdir(train_path):
    if not filename.startswith('.'):
      file_select_train(filename)
      for index in range(0, ((len(df) - (batch_size + 1) * interval) // interval), batch_size):
        x_batch, y_batch = get_batch(batch_size, index)
        train_step(x_batch, y_batch)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)


  for filename in os.listdir(test_path):
    if not filename.startswith('.'):
      file_select_test(filename)
      for index in range(0, ((len(df) - (batch_size + 1) * interval) // interval), batch_size):
        x_batch, y_batch = get_batch(batch_size, index)
        test_step(x_batch, y_batch)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss_test', test_loss.result(), step=epoch)

  # Update the model with the changed weights!
  if epoch % 10 == 0:     
    model.save_weights(checkpoint_prefix)

  template = 'epoch {}, Loss: {}, Test Loss: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         test_loss.result())) 


  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
    
# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

epoch 1, Loss: 252120.96875, Test Loss: 97388.2265625
epoch 2, Loss: 252118.625, Test Loss: 123647.1796875
epoch 3, Loss: 252116.5625, Test Loss: 87516.046875
epoch 4, Loss: 252112.765625, Test Loss: 92066.515625
epoch 5, Loss: 252105.609375, Test Loss: 92453.8984375
epoch 6, Loss: 252109.625, Test Loss: 93532.1640625
epoch 7, Loss: 252117.15625, Test Loss: 167272.15625
epoch 8, Loss: 252105.203125, Test Loss: 168103.671875


KeyboardInterrupt: 

In [ ]:
x,y = get_batch(1,123)
model.predict(x)

In [135]:
y

array([[-2.15240879]])

In [12]:
%tensorboard --logdir /Users/amitaflalo/Desktop/deepnav/logs


Reusing TensorBoard on port 6006 (pid 11502), started 3 days, 1:22:34 ago. (Use '!kill 11502' to kill it.)